# Odstranjevanje šuma z globoko nevronsko mrežo

Naloga zajema generiranje učnih slik, ki vsebujejo naključno barvo ozadja in razne elemente (trikotnike, štirikotnike, elipse in zvezde), dodajanje šuma v generirane učne slike, uporabo učnih slik za učenje nevronske mreže za odstranjevanje šuma in testiranje nevronske mreže z novimi generiranimi slikami in resničnimi fotografijami.

## 1. Predstavitev pomožnih funkcij

V spodnjem odseku so vsi uporabljeni uvoženi paketi, knjižnice in globalne spremenljivke. 

Nevronska mreža se je učila na grafični kartici.

In [ ]:
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as pyplot
import matplotlib.image as mpimg
import random
import math
import torch
import os
import torchvision
from nn.nn import NeuralNetwork

height = 256
width = 512
noise_types = ["mul", "add"]
std_deviation = 0.3
epochs = 10
images_to_generate = 1000
images_to_test = 1000
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

print(f"Device: {device}")

Funkcija za generiranje naključnega števila med parametroma `min` in `max`.

In [ ]:
def random_number(min, max):
  return random.randint(math.floor(min), math.floor(max))

Funkcija za nalaganje vseh slik iz datoteke, uporabljeno za testiranje nevronske mreže z realnimi fotografijami.

In [ ]:
def load_images_from_folder(folder):
  images = []
  for filename in os.listdir(folder):
      img = cv2.imread(os.path.join(folder,filename))
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      if img is not None:
          images.append(img)
  return images

Pomožna funkcija za prikaz slike.

In [ ]:
def display_image(image, cmap="viridis"):
  imgplot = pyplot.imshow(image, cmap=cmap)
  pyplot.axis('off')
  pyplot.show()

Prva veja nevronske mreže se uči na RGB kanalih slike, ta pomožna funkcija pa služi za razdelitev slike na tri kanale, in sicer na rdeč, zelen in moder kanal (RGB).

In [ ]:
def get_rgb(image):
  image_r = image[:,0:1,:,:]
  image_g = image[:,1:2,:,:]
  image_b = image[:,2:3,:,:]
  return image_r, image_g, image_b

Tenzorji morajo biti v obliki [N, channels, height, width], ta funkcija pa služi, da sliki doda novo dimenzijo in dimenzije transponira v obliko, ki je bila opisana.

In [ ]:
def expand(image): 
  expanded = image.transpose(-1, 0, 1)
  expanded = np.expand_dims(expanded, axis = 0)
  return expanded

Slike so bile normalizirane na vrednosti na intervalu [0, 1], vendar se v določenih primerih (npr. dodajanju šuma) lahko zgodi, da bodo vrednosti izven tega intervala. V tem primeru uporabimo pomožno funkcijo, ki vrednosti večje od 1 postavi na 1, vrednosti manjše od 0 pa postavi na 0.

In [ ]:
def fix_image(image):
  image[image > 1] = 1
  image[image < 0] = 0

## 2. Generiranje oblik

V naslednjih štirih odsekih kode se nahajajo funkcije za generiranje trikotnikov, štirikotnikov, elips in zvezd.

V vseh štirih primerih se generirajo naključno postavljene točke in tri naključna števila na intervalu [0, 255] za naključno barvo.

### 2.1 Generiranje trikotnikov

Za generiranje trikotnikov se zgenerirajo tri naključne točke, ki se z uporabo `fillPoly` povežejo, med njimi pa se prostor zapolni z naključno generirano barvo.

In [ ]:
def create_triangle(rand_height_start, rand_height_end, rand_width_start, rand_width_end, image):
  ppt = np.array([
    [random_number(rand_width_start, rand_width_end), random_number(rand_height_start, rand_height_end)], 
    [random_number(rand_width_start, rand_width_end), random_number(rand_height_start, rand_height_end)], 
    [random_number(rand_width_start, rand_width_end), random_number(rand_height_start, rand_height_end)], 
  ], np.int32)

  ppt = ppt.reshape((-1, 1, 2))
  cv2.fillPoly(image, [ppt], (random_number(0, 255), random_number(0, 255), random_number(0, 255)), 8)

### 2.2 Generiranje štirikotnikov

Podobno kot za generiranje trikotnikov, vendar da se tukaj zgenerirajo štiri naključne točke.

In [ ]:
def create_rectangle(rand_height_start, rand_height_end, rand_width_start, rand_width_end, image):
  ppt = np.array([
    [random_number(rand_width_start, rand_width_end), random_number(rand_height_start, rand_height_end)], 
    [random_number(rand_width_start, rand_width_end), random_number(rand_height_start, rand_height_end)], 
    [random_number(rand_width_start, rand_width_end), random_number(rand_height_start, rand_height_end)], 
    [random_number(rand_width_start, rand_width_end), random_number(rand_height_start, rand_height_end)], 
  ], np.int32)

  ppt = ppt.reshape((-1, 1, 2))
  cv2.fillPoly(image, [ppt], (random_number(0, 255), random_number(0, 255), random_number(0, 255)), 8)

### 2.3 Generiranje elips

V primeru generiranja elips, pa se ustvari naključna točka, ki služi za središče in velikost elipse. Naključno se tudi pridobi kot na intervalu [0, 360], začetni in končni kot pa sta postavljena na 0 in 360 stopinj. Pridobi se naključna barva, parameter `-1` pa služi, da se elipsa zapolni z prej omenjeno barvo.

In [ ]:
def create_ellipse(height, width, image):
  cv2.ellipse(
    image,
    (random_number(0, width), random_number(0, height)),
    (random_number(0, width), random_number(0, height)),
    random_number(0, 360),
    0,
    360,
    (random_number(0, 255), random_number(0, 255), random_number(0, 255)),
    -1,
    8
  )

### 2.4 Generiranje zvezd

Pri generiranju zvezd se ustvari 5 naključnih točk in se zračuna njihovo središče. Vsaka točka je nato povezana s tem središčem, tej črti pa je dodeljena naključna barva.

In [ ]:
def create_stars(height, width, image):
  points = []
  for i in range(5):
    points.append((random_number(0, width), random_number(0, height)))

  center = np.array(points).mean(axis = 0)

  for point in points:
    cv2.line(
      image,
      point,
      (math.floor(center[0]), math.floor(center[1])),
      (random_number(0, 255), random_number(0, 255), random_number(0, 255)),
      2,
      8
    )

## 3. Generiranje slike

Ustvarijo se tri dvodimenzionalna polja velikosti (height x width), vsako polje ima eno naključno vrednost. Te tri polja se konkatenirajo, da se pridobi tridimenzionalno polje oblike [height, width, 3]. Na to polje se v naključnem vrstnem redu dodajajo elementi (trikotniki, štirikotniki, elipse in zvezde).

In [ ]:
def generate_image(height, width, triangles, rectangles, ellipses, stars):
  r = np.full((height, width), random_number(0, 255))
  g = np.full((height, width), random_number(0, 255))
  b = np.full((height, width), random_number(0, 255))

  rand_height_start = height * -0.1
  rand_height_end = height * 1.1 
  rand_width_start = width * -0.1
  rand_width_end = width * 1.1 

  image = np.dstack((r, g, b))
  shapes = [triangles, rectangles, ellipses, stars]
  index = random_number(0, 3)
  while sum(shapes) > 0:
    while(shapes[index] == 0):
      index = random_number(0, 3)
    
    if(index == 0):
      create_triangle(rand_height_start, rand_height_end, rand_width_start, rand_width_end, image)
    elif(index == 1):
      create_rectangle(rand_height_start, rand_height_end, rand_width_start, rand_width_end, image)
    elif(index == 2):
      create_ellipse(height, width, image)
    elif(index == 3):
      create_stars(height, width, image)

    shapes[index] -= 1

  return image

## 4. Dodajanje šuma

Pri dodajanju šuma sta dve možnosti, in sicer dodajanje aditivnega in dodajanje multiplikativnega šuma. V primeru aditivnega šuma se šum pridobi s pomočjo Gaussove distribucije s povprečjem 0 in standardnim odklonom, kot je definiran zgoraj v globalnih spremenljivkah., v primeru multiplikativnega šuma pa je enak postopek, le da je povprečje 1. Dobljen šum se potem prišteje ali zmnoži (odvisno od tipa šuma) z vhodno sliko.

In [ ]:
def add_noise(image, type):
  if type == "add":
    noise = np.random.normal(0, std_deviation, image.shape)
    image = image + noise
  elif type == "mul":
    noise = np.random.normal(1, std_deviation, image.shape)
    image = image * noise

  return image

## 5. Učenje nevronske mreže

Nevronska mreža je sestavljena iz dveh vej, zgradba pa je definirana v datotekah `nn.py`, `resnet.py` in `secondbranch.py`.

**Resnet:**
* V primeru prvega zaporednega resneta je prvi korak `Conv2d(3, 32, kernel_size=(3, 3), padding="same")`, v nasprotnem primeru pa `Conv2d(32, 32, kernel_size=(3, 3), padding="same")`
* Nato sledijo koraki `BatchNorm2D`, `Dropout2D` in `ReLU` aktivacijska funkcija
* Naslednji korak je ponovno Conv2D, in sicer `Conv2d(32, 32, kernel_size=(3, 3), padding="same")`
* Ponovno sledita koraka `BatchNorm2D` in `Dropout2D`
* Na koncu se seštejeta vhodni tenzor in tenzor z vsemi koraki in se uporabi aktivacijska funkcija `ReLU`

**Prva veja:**
* Na vsakem izmed kanalov (R, G, B) se izvede `Conv2d(1, 8, kernel_size=(11, 11), padding="same", bias=False)`

**Druga veja:**
* Izvedejo se trije zaporedni Resnet bloki
* Sledi `Conv2d(32, 8, kernel_size=(1, 1), padding="same")`
* In na koncu `Softmax(dim=1)`

**Združitev:**
* Izhod iz prve veje so trije tenzorji, izhod iz druge veje pa en tenzor
* Z vsakim tenzorjem iz prve veje se zmnoži drugi tenzor
* Vsak izmed treh tenzorjev ima drugo dimenzijo `8`, ta dimenzija se sešteje v eno
* Dobljeni so trije tenzorji oblike [N, 1, H, W], ki se konkatenirajo v [N, 3, H, W]

In [ ]:
def train(noised, originals, model, optimizer, loss_fn, epochs):
  model = model.to(device)

  model.train()

  for epoch in range(epochs):
    print('epoch:', epoch)
    for step in range(len(noised)):
      if step % 100 == 0:
        print('training image:', step)
      image = torch.from_numpy(noised[step:step+1])

      #print(image[0].shape)
      #display_image(image[0].cpu().detach().numpy().transpose(1, 2, 0))

      image_r, image_g, image_b = get_rgb(image)

      #test_img = torch.cat((image_r, image_g, image_b), dim = 1)
      #print(test_img.shape)

      #display_image(test_img[0].cpu().detach().numpy().transpose(1, 2, 0))

      original = torch.from_numpy(originals[step:step+1])

      image = image.to(device)
      image_r = image_r.to(device)
      image_g = image_g.to(device)
      image_b = image_b.to(device)

      original = original.to(device)

      #print(image.shape)
      #print(image_r.shape)
      #print(image_g.shape)
      #print(image_b.shape)
      pred = model(image, image_r, image_g, image_b)

      loss = loss_fn(pred[0, :, :, :], original[0, :, :, :])

      optimizer.zero_grad()

      loss.backward()

      optimizer.step()

## 6.1 Preizkuševanje nevronske mreže z generiranimi slikami

V tej funkciji se naučena nevronska mreža preizkusi z naključno generiranimi slikami.

In [ ]:
def test_generated(model, noise_type):
  std_dev_original = 0
  std_dev_denoised = 0
  mse = 0

  model = model.to(device)
  model.eval()
  
  for i in range(images_to_test):
    print('testing image:', i)
    with torch.no_grad():
      image = generate_image(height, width, 3, 3, 3, 3)
        
      image = image / 255

      if i == 0:
        display_image(image)

      image = add_noise(image, noise_type)
      fix_image(image)

      if i == 0:
        display_image(image)

      images = expand(image)

      images = images.astype(np.float32)

      images = torch.from_numpy(images).to(device)
      r, g, b = get_rgb(images)

      r = r.to(device)
      g = g.to(device)
      b = b.to(device)

      pred = model(images, r, g, b)
      
      fix_image(pred)

      pred_img = pred[0, :, :, :].cpu().detach().numpy()
      pred_img = pred_img.transpose(1, 2, 0)
      #fix_image(pred_img)
      
      if i == 0:
        display_image(pred_img)

      mse += np.square(np.subtract(image, pred_img)).mean()
      std_dev_denoised += torch.std(torch.from_numpy(pred_img))
      std_dev_original += torch.std(torch.from_numpy(image))

  std_dev_original /= images_to_test
  std_dev_denoised /= images_to_test

  mse /= images_to_test
  psnr = 20 * math.log10(255/math.sqrt(mse))
  snr = 10 * math.log10(std_dev_original/std_dev_denoised)

  print("PSNR: ", psnr)
  print("SNR: ", snr)

## 6.2 Preizkuševanje nevronske mreže z resničnimi fotografijami

V tej funkciji pa se naučena nevronska mreža preizkusi z resničnimi fotografijami, ki so naložene iz diska.

In [ ]:
def test_from_disk(model):
  loaded_images = load_images_from_folder("assets")

  for image in loaded_images:
    h, w, c = image.shape
    image = image / 255

    display_image(image)

    images = expand(image)

    images = images.astype(np.float32)

    images = torch.from_numpy(images).to(device)
    
    r = images[:,0,:,:].reshape(1, 1, h, w).to(device)
    g = images[:,1,:,:].reshape(1, 1, h, w).to(device)
    b = images[:,2,:,:].reshape(1, 1, h, w).to(device)

    pred = model(images, r, g, b)

    pred_img = pred[0, :, :, :].cpu().detach().numpy()
    pred_img = pred_img.transpose(1, 2, 0)
    fix_image(pred_img)

    display_image(pred_img)

## 7. Generiranje slik

Še zadnja pomožna funkcija, ki zgenerira N naključnih slik in jim doda aditivni ali multiplikativni šum, ter jim spremeni obliko v [N, 3, H, W]

In [ ]:
def get_images(noise_type):
  originals = np.empty((images_to_generate, height, width, 3))
  noised = np.empty((images_to_generate, height, width, 3))
  for i in range(images_to_generate):
    image = generate_image(height, width, 3, 3, 3, 3)
    image = image / 255

    originals[i] = image

    noised_image = add_noise(image, noise_type)
    fix_image(noised_image)

    noised[i] = noised_image

  fig, ax = pyplot.subplots(1, 2)

  ax[0].set_title(f'{1}')
  ax[0].imshow(originals[0, :, :, :])
  ax[0].set_axis_off()

  ax[1].set_title(f'{2}')
  ax[1].imshow(noised[0, :, :, :])
  ax[1].set_axis_off()

  #originals = np.reshape(originals, (images_to_generate, height, width, 3))
  originals = np.transpose(originals, (0, 3, 1, 2))

  #noised = np.reshape(noised, (images_to_generate, height, width, 3))
  noised = np.transpose(noised, (0, 3, 1, 2))

  #print(originals.shape)
  #print(noised.shape)

  #test_org = originals[0, :, :, :].transpose(1, 2, 0)
  #test_noised = noised[0, :, :, :].transpose(1, 2, 0)

  #print(test_org.shape)
  #print(test_noised.shape)

  #display_image(test_org)
  #display_image(test_noised)

  return originals, noised

## 8. Prikaz naključno generiranih slik

V spodnjem izpisu so prikazane naključno generirane slike v štirih vrstah:
* prva vrsta - naključne slike s tremi trikotniki
* druga vrsta - naključne slike s tremi štirikotniki
* tretja vrsta - naključne slike s tremi elipsami
* četrta vrsta - naključne slike s tremi zvezdami
* peta vrsta - naključne slike s tremi izmed vsakega elementa

In [ ]:
for i in range(5):
  array = [0, 0, 0, 0]

  if i == 4:
    array = [3, 3, 3, 3]
  else:
    array[i] = 3

  generated_images = np.empty((5, 256, 256, 3))
  for j in range(5):
    generated_images[j] = generate_image(256, 256, array[0], array[1], array[2], array[3])
    generated_images[j] /= 255
    generated_images[generated_images > 1] = 1
    generated_images[generated_images < 0] = 0

  fig, ax = pyplot.subplots(1, 5)
  for n in range(5):
    ax[n].set_title(f'{n + 1}')
    ax[n].imshow(generated_images[n, :, :, :])
    ax[n].set_axis_off()

## 9. Prikaz učenja in testiranja

Spodnja for zanka služi, da se nevronska mreža prvo nauči za aditivni šum, nato pa za multiplikativni šum.

Spodnji izpisi prikazujejo naključno generirano sliko z in brez šuma, nato pa učenje in testiranje na 1000 naključnih slikah, kjer se prikaže rezultat le za prvo sliko. Na koncu pa še sledi testiranje na vseh slikah, naloženih iz datoteke.

In [ ]:
for noise_type in noise_types:
  print('------------------------')
  print('NOISE TYPE:', noise_type)
  
  print('generating images...')

  originals, noised = get_images(noise_type)

  print('generated images')

  model = NeuralNetwork()
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
  loss_fn = torch.nn.MSELoss()

  model = model.to(device)

  noised = noised.astype(np.float32)
  originals = originals.astype(np.float32)

  print('training...')

  train(noised, originals, model, optimizer, loss_fn, epochs)

  print('done training')

  torch.save(model, "denoising.pt")

  filters = model.filter_layer.weight.cpu().detach().numpy()
  N = filters.shape[0]

  fig, ax = pyplot.subplots(1, N)
  for n in range(N):
    ax[n].set_title(f'{n}')
    ax[n].imshow(filters[n, 0], cmap='gray')
    ax[n].set_axis_off()

  test_generated(model, noise_type)
  test_from_disk(model)